<a href="https://www.kaggle.com/code/aaradhyakulkarni/flood-prediction-competition?scriptVersionId=176859206" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# df_train = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
# df_test = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')


In [ ]:
df_train.shape , df_test.shape

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.drop(['id'], axis=1, inplace=True)

In [ ]:
df_train.info()

In [ ]:
df_train.isna().sum()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
sns.histplot(df_train['FloodProbability'])

In [ ]:
numerical_columns = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']

In [ ]:


for column in numerical_columns:
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    sns.histplot(data=df_train, x=column, ax=axs[0])
    axs[0].set_title(f'Histogram of {column}')

    sns.boxplot(data=df_train, y=column, ax=axs[1])
    axs[1].set_title(f'Boxplot of {column}')

    plt.tight_layout()
    plt.show()

In [ ]:
X = df_train.drop(['FloodProbability'], axis=1)
y = df_train['FloodProbability']


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled.head()

In [ ]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
#decision tree
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

print('Decision Tree Regressor')
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('R2 Score:', r2_score(y_test, y_pred))


In [ ]:
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)
xgb_mse = mean_squared_error(y_test, xgb_preds)
xgb_r2 = r2_score(y_test, xgb_preds)

print(f'XGBRegressor MSE: {xgb_mse}')
print(f'XGBRegressor R2 Score: {xgb_r2}')

In [ ]:
#predicting on test data
X_test = df_test.drop(['id'], axis=1)
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
X_test_scaled.head()

In [ ]:
xgb1 = XGBRegressor()
xgb1.fit(X_scaled, y)
y_pred = xgb1.predict(X_test_scaled)

In [ ]:
#Submission file
submission = pd.DataFrame({'id': df_test['id'], 'FloodProbability': y_pred})
submission.to_csv('submission.csv', index=False)
